In [1]:
!pip install --upgrade mplfinance

     |████████████████████████████████| 62 kB 620 kB/s eta 0:00:011


In [234]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime
import glob
import json
import mplfinance as mpf

In [235]:
def convert_sec_to_min(date):
    split_date = date.split(':')
    min_date = split_date[0]+':'+split_date[1]+':00'
    return min_date


def get_data(stock_name='AAPL', start='2015-01-01', end='2015-12-25',interval= '1m'):
    stock_names = stock_name.split()
    df = yf.download(stock_name, start=start, end=end, group_by='ticker', interval = interval)
    return df
def get_trade_signal(trading_date, price,direction):
    import numpy as np
    signal   = []
    previous = -1.0
    factor = 0.99 if direction == 'buy' else 1.01
    new_trade = []
    for i in trading_date:
        continue_flag = 1
        ind = 0 
      #  import pdb;pdb.set_trace()
        while continue_flag:
            try:
               print(price.loc[i])
               new_trade.append(i)
               continue_flag = 0
            except Exception as e:
               print('can not find the date in k line',e)
               ind +=1
               i = str(datetime.datetime.strptime(i,"%Y-%m-%d %H:%M:%S") + datetime.timedelta(minutes=1))
              
            if ind > 10:
                continue_flag = 0
    if len(new_trade) == 0:
       for i in trading_date:
        continue_flag = 1
        ind = 0 
      #  import pdb;pdb.set_trace()
        while continue_flag:
            try:
               print(price.loc[i])
               new_trade.append(i)
               continue_flag = 0
            except Exception as e:
               print('can not find the date in k line',e)
               ind +=1
               i = str(datetime.datetime.strptime(i,"%Y-%m-%d %H:%M:%S") - datetime.timedelta(minutes=1))
              
            if ind > 10:
                continue_flag = 0
        

    for date,value in price.iteritems():
        tem_date = str(date).split('-04')[0]
        if  tem_date in new_trade:
            print(date)
            signal.append(price[date]*factor)

        else:
            signal.append(np.nan)
        previous = value
    return signal

def get_stock_price_info(ticker,period = '1y',interval = '1d',):
   data = yf.download(  # or pdr.get_data_yahoo(...
        # tickers list or string as well
        tickers = ticker,

        # use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
        period = period,

        # fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
        interval = interval,

        # group by ticker (to access via data['SPY'])
        # (optional, default is 'column')
        group_by = 'ticker',

        # adjust all k line automatically
        # (optional, default is False)
        auto_adjust = True,

        # download pre/post regular market hours data
        # (optional, default is False)
        prepost = False,

        # use threads for mass downloading? (True/False/Integer)
        # (optional, default is True)
        threads = True,

        # proxy URL scheme use use when downloading?
        # (optional, default is None)
        proxy = None
    )
   return data   

In [236]:
def draw_trading_log(data, symbol, end):
    trading_log = {}
    for i in range(5):
        try:
            start = str( end - datetime.timedelta(days=i))
            with open(log_save_path+ start+'/'+symbol+'.json') as f:       
                 data = json.load(f)
            trading_log.update(data)
        except Exception as e:
            print(e)

    buy_date = [convert_sec_to_min(i) for i in trading_log.keys() if trading_log[i]['Direction'] == 'buy']
    sell_date = [convert_sec_to_min(i) for i in trading_log.keys() if trading_log[i]['Direction'] == 'sell']        
    
    buy_signal = get_trade_signal(buy_date,[],week_data['Close'])
    sell_signal = get_trade_signal([],sell_date,week_data['Close'])

    apds = [ 
         mpf.make_addplot(buy_signal,type='scatter',markersize=200,marker='^'),
         mpf.make_addplot(sell_signal,type='scatter',markersize=200,marker='v'),
       ]
                 
                 
    mpf.plot(week_data,warn_too_much_data=2000,volume=True,addplot=apds)

    
    

In [237]:
def split_buy_sell_date(trading_log):
    sell_date = []
    buy_date = []
    for i in trading_log.keys():
        if i not in ['open', 'close', 'pre_open', 'pre_close'] :
           if trading_log[i]['Direction'] == 'buy' :
                buy_date.append(convert_sec_to_min(i))
           else:
                sell_date.append(convert_sec_to_min(i)) 
    return buy_date, sell_date



In [242]:


end = datetime.date.today() 

start = end -  datetime.timedelta(days=7)   
trading_stocks = [ 'AN', 'AVLR', 'CHGG', 'CMG', 'CPA', 'CRWD', 'DE', 'ELAN', 'EVR', 'FIVN', 'GO', 'HD', 'LPLA', 'PAG', 'PD', 'QRTEA', 'RNG', 'RYN', 'TTC', 'WY']
for symbol in list(trading_stocks):

    print(symbol)
    week_data = pd.read_csv('https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol='+symbol+'&interval='+interval+'&slice='+time+'&apikey=H40LS6V3016EZSQ6',index_col=False)
    week_data = week_data[::-1].reset_index()
    week_data.index = pd.to_datetime(week_data.time)
    week_data  = week_data[start : ]
    #week_data = get_data(stock_name=symbol, start=start, end=end)
    trading_log = {}
    mpf.plot(week_data,warn_too_much_data=3000,volume=True, title = symbol,
                  savefig=dict(fname=symbol+'.jpg',dpi=100,pad_inches=0.25))
    print('save the image', symbol )

import glob
import img2pdf
with open(str(end)+"_stock_pool.pdf","wb") as f:

         f.write(img2pdf.convert([i+".jpg" for i in trading_stocks]))

AN
save the image AN
AVLR
save the image AVLR
CHGG
save the image CHGG
CMG
save the image CMG
CPA
save the image CPA
CRWD
save the image CRWD
DE
save the image DE
ELAN
save the image ELAN
EVR
save the image EVR
FIVN
save the image FIVN
GO
save the image GO
HD
save the image HD
LPLA
save the image LPLA
PAG
save the image PAG
PD
save the image PD
QRTEA
save the image QRTEA
RNG
save the image RNG
RYN
save the image RYN
TTC
save the image TTC
WY
save the image WY


In [241]:


end = datetime.date.today() 

log_save_path='/mnt/c/Users/zhubo/Documents/trading_data/'
trading_stocks = []
for i in range(7):
    start = str( end - datetime.timedelta(days=i))
    trading_stocks += glob.glob(log_save_path+ start+'/*.json')
trading_stocks = set([ i.split('/')[-1].split('.')[0] for i in trading_stocks])
start = end -  datetime.timedelta(days=7)   

for symbol in list(trading_stocks):

    print(symbol)
    week_data = pd.read_csv('https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol='+symbol+'&interval='+interval+'&slice='+time+'&apikey=H40LS6V3016EZSQ6',index_col=False)
    week_data = week_data[::-1].reset_index()
    week_data.index = pd.to_datetime(week_data.time)
    week_data  = week_data[start : ]
   # week_data = get_data(stock_name=symbol, start=start, end=end)
    trading_log = {}
    for i in range(7):
        try:
            start = str( end - datetime.timedelta(days=i))
            print(start)
            with open(log_save_path+ start+'/'+symbol+'.json') as f:       
                 data = json.load(f)
            trading_log.update(data)
        except Exception as e:
            pass
    print('trading log', trading_log)
    
    buy_date, sell_date = split_buy_sell_date(trading_log)     
    
    buy_signal = get_trade_signal(buy_date,week_data['close'], 'buy')
    sell_signal = get_trade_signal(sell_date,week_data['close'], 'sell')

    apds = []
    if sum(~np.isnan(buy_signal)) >0:
         apds.append(mpf.make_addplot(buy_signal,type='scatter',markersize=200,marker='^'))
    if sum(~np.isnan(sell_signal)) >0:
         apds.append(mpf.make_addplot(sell_signal,type='scatter',markersize=200,marker='v'))
    if len(apds) > 0:

       mpf.plot(week_data,warn_too_much_data=2000,volume=True,addplot=apds, title = symbol,
                  savefig=dict(fname=symbol+'.jpg',dpi=100,pad_inches=0.25))
       print('save the image')
    else:
       mpf.plot(week_data,warn_too_much_data=2000,volume=True, title = symbol,
                  savefig=dict(fname=symbol+'.jpg',dpi=100,pad_inches=0.25))
       print('save the image', symbol )

import glob
import img2pdf
with open(str(start)+'_'+str(end)+"_trading_log.pdf","wb") as f:
    f.write(img2pdf.convert(glob.glob("*.jpg")))

FLL
2021-06-12
2021-06-11
2021-06-10
2021-06-09
2021-06-08
2021-06-07
2021-06-06
trading log {'2021-06-11 11:17:00': {'Symbol': 'FLL', 'Price': 9.483426706136083, 'Direction': 'sell', 'Size': 1.0, 'Position': 0}, '2021-06-07 12:22:00': {'Symbol': 'FLL', 'Price': 10.98, 'Direction': 'buy', 'Size': 1, 'Position': 0, 'stop_loss_range': 9.483151791473807, 'stop_earning_range': 11.866848208526193}}
10.96
2021-06-07 12:22:00
9.55
2021-06-11 11:17:00
save the image
PD
2021-06-12
2021-06-11
2021-06-10
2021-06-09
2021-06-08
2021-06-07
2021-06-06
trading log {'2021-06-08 15:45:00': {'Symbol': 'PD', 'Price': 38.64, 'Direction': 'buy', 'Size': 1, 'Position': 0, 'stop_loss_range': 32.23818724358395, 'stop_earning_range': 41.72181275641605}, '2021-06-08 13:45:00': {'Symbol': 'PD', 'Price': 38.64, 'Direction': 'buy', 'Size': 1, 'Position': 0, 'stop_loss_range': 32.23818724358395, 'stop_earning_range': 41.72181275641605}, '2021-06-08 15:54:00': {'Symbol': 'PD', 'Price': 38.86, 'Direction': 'sell', 'Si

save the image STKS
SE
2021-06-12
2021-06-11
2021-06-10
2021-06-09
2021-06-08
2021-06-07
2021-06-06
trading log {'2021-06-07 15:17:00': {'Symbol': 'SE', 'Price': 256.16, 'Direction': 'buy', 'Size': 1, 'Position': 0, 'stop_loss_range': 238.52410385926592, 'stop_earning_range': 269.63589614073413}, '2021-06-07 15:59:00': {'Symbol': 'SE', 'Price': 256.16, 'Direction': 'buy', 'Size': 1, 'Position': 0, 'stop_loss_range': 239.3252779313631, 'stop_earning_range': 268.8347220686369}}
256.86
257.12
2021-06-07 15:17:00
2021-06-07 15:59:00


/home/zhubo/.local/lib/python3.8/site-packages/mplfinance/_arg_validators.py:35: UserWarning: 


            POSSIBLE TO SEE DETAILS (Candles, Ohlc-Bars, Etc.)
   For more information see:
   - https://github.com/matplotlib/mplfinance/wiki/Plotting-Too-Much-Data
   
   TO SILENCE THIS WARNING, set `type='line'` in `mpf.plot()`
   OR set kwarg `warn_too_much_data=N` where N is an integer 
   LARGER than the number of data points you want to plot.

  warnings.warn('\n\n ================================================================= '+


save the image
RH
2021-06-12
2021-06-11
2021-06-10
2021-06-09
2021-06-08
2021-06-07
2021-06-06
trading log {'2021-06-10 09:37:00': {'Symbol': 'RH', 'Price': 628.92, 'Direction': 'buy', 'Size': 1, 'Position': 0, 'stop_loss_range': 589.0832593156294, 'stop_earning_range': 652.7067406843705}, '2021-06-07 12:08:00': {'Symbol': 'RH', 'Price': 618.45, 'Direction': 'buy', 'Size': 1, 'Position': 0, 'stop_loss_range': 588.1435746409979, 'stop_earning_range': 643.4764253590021}, '2021-06-07 15:59:00': {'Symbol': 'RH', 'Price': 618.45, 'Direction': 'buy', 'Size': 1, 'Position': 0, 'stop_loss_range': 589.5266761870878, 'stop_earning_range': 642.0933238129122}}
696.12
can not find the date in k line '2021-06-07 12:08:00'
624.51
624.83
2021-06-07 12:09:00
2021-06-07 15:59:00
2021-06-10 09:37:00
save the image
FIVN
2021-06-12
2021-06-11
2021-06-10
2021-06-09
2021-06-08
2021-06-07
2021-06-06
trading log {'2021-06-09 12:39:00': {'Symbol': 'FIVN', 'Price': 160.75, 'Direction': 'buy', 'Size': 1, 'Positio

/home/zhubo/.local/lib/python3.8/site-packages/mplfinance/_arg_validators.py:35: UserWarning: 


            POSSIBLE TO SEE DETAILS (Candles, Ohlc-Bars, Etc.)
   For more information see:
   - https://github.com/matplotlib/mplfinance/wiki/Plotting-Too-Much-Data
   
   TO SILENCE THIS WARNING, set `type='line'` in `mpf.plot()`
   OR set kwarg `warn_too_much_data=N` where N is an integer 
   LARGER than the number of data points you want to plot.

  warnings.warn('\n\n ================================================================= '+


...

save the image
NIO
2021-06-12
2021-06-11
2021-06-10
2021-06-09
2021-06-08
2021-06-07
2021-06-06
trading log {'2021-06-07 11:12:00': {'Symbol': 'NIO', 'Price': 40.9, 'Direction': 'buy', 'Size': 1, 'Position': 0, 'stop_loss_range': 39.10174442278927, 'stop_earning_range': 43.18825557721073}, '2021-06-07 15:59:00': {'Symbol': 'NIO', 'Price': 40.9, 'Direction': 'buy', 'Size': 1, 'Position': 0, 'stop_loss_range': 38.99070601944456, 'stop_earning_range': 43.29929398055544}}
43.172
43.6363
2021-06-07 11:12:00
2021-06-07 15:59:00


/home/zhubo/.local/lib/python3.8/site-packages/mplfinance/_arg_validators.py:35: UserWarning: 


            POSSIBLE TO SEE DETAILS (Candles, Ohlc-Bars, Etc.)
   For more information see:
   - https://github.com/matplotlib/mplfinance/wiki/Plotting-Too-Much-Data
   
   TO SILENCE THIS WARNING, set `type='line'` in `mpf.plot()`
   OR set kwarg `warn_too_much_data=N` where N is an integer 
   LARGER than the number of data points you want to plot.

  warnings.warn('\n\n ================================================================= '+


save the image
LOVE
2021-06-12
2021-06-11
2021-06-10
2021-06-09
2021-06-08
2021-06-07
2021-06-06
trading log {'2021-06-09 11:43:00': {'Symbol': 'LOVE', 'Price': 83.86, 'Direction': 'buy', 'Size': 1, 'Position': 0, 'stop_loss_range': 75.53519040882765, 'stop_earning_range': 89.68480959117235}, '2021-06-10 15:59:00': {'Symbol': 'LOVE', 'Price': 85.52, 'Direction': 'buy', 'Size': 1, 'Position': 0, 'stop_loss_range': 75.53892794217876, 'stop_earning_range': 91.74107205782124}, '2021-06-07 13:03:00': {'Symbol': 'LOVE', 'Price': 85.01, 'Direction': 'buy', 'Size': 1, 'Position': 0, 'stop_loss_range': 77.08991516654261, 'stop_earning_range': 91.0700848334574}, '2021-06-03 10:55:00': {'Symbol': 'LOVE', 'Price': 85.67, 'Direction': 'buy', 'Size': 1, 'Position': 0, 'stop_loss_range': 80.46426732232388, 'stop_earning_range': 90.70573267767612}}
86.81
87.49
85.82
can not find the date in k line '2021-06-03 10:55:00'
can not find the date in k line '2021-06-03 10:56:00'
can not find the date in k li

/home/zhubo/.local/lib/python3.8/site-packages/mplfinance/_arg_validators.py:35: UserWarning: 


            POSSIBLE TO SEE DETAILS (Candles, Ohlc-Bars, Etc.)
   For more information see:
   - https://github.com/matplotlib/mplfinance/wiki/Plotting-Too-Much-Data
   
   TO SILENCE THIS WARNING, set `type='line'` in `mpf.plot()`
   OR set kwarg `warn_too_much_data=N` where N is an integer 
   LARGER than the number of data points you want to plot.

  warnings.warn('\n\n ================================================================= '+


save the image
ANVS
2021-06-12
2021-06-11
2021-06-10
2021-06-09
2021-06-08
2021-06-07
2021-06-06
trading log {'2021-06-03 11:03:00': {'Symbol': 'ANVS', 'Price': 57.26, 'Direction': 'buy', 'Size': 1, 'Position': 0, 'stop_loss_range': 42.01562262981844, 'stop_earning_range': 70.04437737018155}, '2021-06-03 15:59:00': {'Symbol': 'ANVS', 'Price': 57.26, 'Direction': 'buy', 'Size': 1, 'Position': 0, 'stop_loss_range': 41.53655389526889, 'stop_earning_range': 70.52344610473111}}
can not find the date in k line '2021-06-03 11:03:00'
can not find the date in k line '2021-06-03 11:04:00'
can not find the date in k line '2021-06-03 11:05:00'
can not find the date in k line '2021-06-03 11:06:00'
can not find the date in k line '2021-06-03 11:07:00'
can not find the date in k line '2021-06-03 11:08:00'
can not find the date in k line '2021-06-03 11:09:00'
can not find the date in k line '2021-06-03 11:10:00'
can not find the date in k line '2021-06-03 11:11:00'
can not find the date in k line '202